In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
import base64
from openai import OpenAI
import fitz  # PyMuPDF
from pathlib import Path

In [15]:
client = OpenAI()
downloads = Path("./../downloads")
pdfs = list((downloads / "pdfs").iterdir())
images = downloads / "images"
dishes = [p.name.replace(".pdf", "") for p in pdfs]

In [13]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def pdf2image(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    image_pth = images / pdf_path.name.replace(".pdf", "")
    image_pth.mkdir()
    # Iterate through each page and save it as an image
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        pix = page.get_pixmap()
        image_path = image_pth / f'page_{page_num + 1}.png'
        pix.save(image_path)

    pdf_document.close()

In [14]:
_ = [pdf2image(p) for p in pdfs]

In [25]:
def generate_ingredients(dish):
  prompt = """You are a helpful AI, your task is to read the recipe and note down all the ingredients for 1 person only in a CSV format with the following headers
  dish_name,name, tags, quantity, metric
  - quantity is always a number
  - metric should be one of {tbsp, tsp, unit, g, cup}
  - tags should be one of {Protein, Pasta, Vegetable, Sauce, Dairy, Spice, Herb, Grain}"""

  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt,
          }
        ] + [
          {
            "type": "image_url",
            "image_url": {
              "detail": "high",
              "url": f"data:image/jpeg;base64,{encode_image(image_path)}",
            },
          } for image_path in (images / dish).iterdir()
        ]
      }
    ],
    max_tokens=300,
  )
  print(response.choices[0].message.content)

In [29]:
for i, d in enumerate(dishes):
    print(f"{i} {d}")

0 bacon-jalapeno-mac-cheese
1 curry-yogurt-chicken
2 fattoush-inspired-salad
3 smart-shrimp-curry
4 beyond-meat-steakhouse-cheeseburgers
5 spicy-tomato-penne-al-forno
6 indian-style-chicken-curry
7 cheesy-pork-enchiladas-and-diy-enchilada-sauce
8 linguine-and-bacon-in-blush-sauce
9 seasoned-shrimp-and-roasted-potatoes
10 honey-garlic-turkey-bowls
11 chipotle-salmon-bowls
12 crispy-parmesan-chicken
13 chorizo-meatball-bowls
14 tuscan-sausage-linguine
15 harissa-sesame-chicken
16 ponzu-glazed-chicken-sandos
17 moroccan-inspired-chicken-thighs
18 chicken-tikka-masala
19 pork-banh-mi-style-bowls
20 cajun-spiced-halloumi-burgers
21 golden-coconut-shrimp
22 persian-inspired-chicken-and-jeweled-rice
23 smoky-barramundi-with-brown-butter
24 chicken-penne-al-limone
25 bbq-black-bean-taquitos


In [37]:
xs = [2, 1, 12, 13, 11, 18, 24, 0, 25, 4, 20, 7]
_ = [generate_ingredients(dishes[x]) for x in xs]

Here are the ingredients for 1 person in CSV format:

```csv
dish_name,name,tags,quantity,metric
Fattoush-Inspired Salad,Chickpeas,Protein,30,g
Fattoush-Inspired Salad,Shawarma Spice Blend,Spice,1, tbsp
Fattoush-Inspired Salad,Garlic,Herb,1,tsp
Fattoush-Inspired Salad,Pita Bread,Grain,1,unit
Fattoush-Inspired Salad,Baby Tomatoes,Vegetable,118,g
Fattoush-Inspired Salad,Parsley,Hull,7.5,g
Fattoush-Inspired Salad,Mixed Olives,Vegetable,30,g
Fattoush-Inspired Salad,Green Onion,Vegetable,2,unit
Fattoush-Inspired Salad,White Wine Vinegar,Dairy,1, tbsp
Fattoush-Inspired Salad,Sweet Bell Pepper,Vegetable,160,g
Fattoush-Inspired Salad,Feta Cheese,Dairy,10,g
Fattoush-Inspired Salad,Sugar,Spice,1,tsp
Fattoush-Inspired Salad,Oil,Dairy,1,tsp
Fattoush-Inspired Salad,Salt and Pepper,Spice,1,tsp
```
Here are the ingredients for one serving of the Curry Yogurt Chicken with Veggie Rice Pilaf in CSV format:

```csv
dish_name,name,tags,quantity,metric
Curry Yogurt Chicken,Chicken Thighs,Protein,142.5,g
Cu